In [10]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [45]:
'''
Cox Rox Rubenstein (European & American)

s - underlying asset price
rf - risk free rate
sigma - volataility
K - strike price
T - time for maturity
n - number of steps
'''

def BinomialCRR(s, rf, sigma, K, T, q=0, n = 70, isEuropean=True): 
    delta = T / n    # single time step
    discount = math.exp((rf - q)  * delta)
    u = math.exp(sigma * math.sqrt(delta))
    d = 1 / u
    p = (discount - d) / (u - d)

    pricing_tree = get_pricing_tree(u, d, s, n + 1)
    if isEuropean:
        option_price = get_european_option_tree(discount, p, n, K, pricing_tree)
    else:
        option_price = get_american_option_tree(discount, p, n, K, pricing_tree)

    return option_price[0, 0]

def get_pricing_tree(u, d, s, n=70):
    pricing_tree = np.zeros((n, n))

    for i in range(n):
        pricing_tree[i, :i+1] = 1
        pricing_tree[i, i+1:] = u**np.arange(1, n-i, 1)
        pricing_tree[i] *= (d ** i)

    pricing_tree = np.multiply(pricing_tree, s)
    pricing_tree = np.triu(pricing_tree)

    return pricing_tree

def get_european_option_tree(discount, p, n, K, pricing_tree):
    option_price = np.zeros((n, n))
    option_price[:, n - 1] = np.maximum(pricing_tree[:, n - 1] - K, 0)

    for i in range(n-1, 1, -1):
        option_price[:, i-1] = option_price[:, i] * p + np.append(option_price[1: , i], 0) * (1 - p)
        option_price[:, i-1] *= discount

    return option_price

def get_american_option_tree(discount, p, n, K, pricing_tree):
    option_price = np.zeros((n, n))
    option_price[:, n - 1] = np.maximum(pricing_tree[:, n - 1] - K, 0)

    for i in range(n-1, 0, -1):
        wait_out = option_price[:, i] * p + np.append(option_price[1: , i], 0) * (1 - p)
        wait_out *= discount
        exercise = option_price[:, i-1] - K
        option_price[:, i-1] = np.maximum(wait_out, exercise)

    return option_price 

In [46]:
'''John C Hull Example 18.3 '''
